In [ ]:
# CPU版FAISSをインストール
!pip install faiss-cpu

!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install pypdf

!pip install unstructured
!pip install unstructured[local-inference]
!pip install "unstructured[pdf]"


# ベクトル化

In [1]:
# ▼ 2. パスの設定
import os

BASE_DIR = os.path.abspath("")  # 現在のノートブックのディレクトリ
DATA_DIR = os.path.join(BASE_DIR, "document")
STORAGE_DIR = os.path.join(BASE_DIR, "storage")

print("DATA_DIR:", DATA_DIR)
print("STORAGE_DIR:", STORAGE_DIR)

os.makedirs(STORAGE_DIR, exist_ok=True)

# ▼ 3. データ読み込み
from langchain_community.document_loaders import UnstructuredFileLoader

target_file = os.path.join(DATA_DIR, "ey-japan-info-sensor-2023-06-03.pdf")

loader = UnstructuredFileLoader(target_file)
documents = loader.load()

print(f" {len(documents)} 件のドキュメントを読み込みました")


# ▼ 4. テキスト分割
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_texts = splitter.split_documents(documents)

print(f"{len(split_texts)} 個のチャンクに分割されました")


# ▼ 5. 埋め込みモデルを読み込む
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
print("埋め込みモデル読み込み完了")

# ▼ 6. FAISSインデックス作成
from langchain_community.vectorstores.faiss import FAISS

index = FAISS.from_documents(documents=split_texts, embedding=embedding_model)
print("FAISSインデックス作成完了")

# ▼ 8. インデックス保存
index.save_local(folder_path=STORAGE_DIR)
print(f"インデックスを {STORAGE_DIR} に保存しました")

DATA_DIR: /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document
STORAGE_DIR: /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/storage


/var/folders/p7/3cyzzk9534gcjt7smq4152xm0000gn/T/ipykernel_98314/3094298852.py:18: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(target_file)
/Users/iwamurahayato/.pyenv/versions/3.12.5/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The PDF <_io.BufferedReader name='/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document/ey-japan-info-sensor-2023-06-03.pdf'> contains a metadata field indicating that it should not allow te

 1 件のドキュメントを読み込みました
10 個のチャンクに分割されました


/var/folders/p7/3cyzzk9534gcjt7smq4152xm0000gn/T/ipykernel_98314/3094298852.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")


埋め込みモデル読み込み完了
FAISSインデックス作成完了
✅ インデックスを /Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/storage に保存しました


## ▼ 7. 検索テスト

In [2]:
query = "固定資産とは何ですか？"
results = index.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"\n--- 類似チャンク {i} ---")
    print(doc.page_content[:200])
    print(f"メタデータ: {doc.metadata}")


--- 類似チャンク 1 ---
土地など使用や時の経過を通じて価値が減少しないものをいい、減損処理を除いては、基本的 に費用配分は行われません。

土地、美術品＊

減耗性資産

採掘あるいは採取されるにつれて漸次減耗し枯渇する天然資源をいい、全体としての用役を もって生産に役立つものではなく、採取されるに応じてその実態が部分的に製品化されるもの です（連続意見書第三 第一 六2参照）。

鉱山や油田における埋蔵 資源、山林


メタデータ: {'source': '/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document/ey-japan-info-sensor-2023-06-03.pdf'}

--- 類似チャンク 2 ---
交換

自己所有の固定資産と交換に固定資産を取得した場合には、交換に供された自己資産の適正な簿価をもって取得原価とする。 自己所有の株式ないし社債等と固定資産を交換した場合には、当該有価証券の時価又は適正な簿価をもって取得原価とする＊2。

贈与

固定資産を贈与された場合には、時価等を基準として公正に評価した額をもって取得原価とする。

＊1　なお、企業会計基準第8号「ストック・オプション等に関
メタデータ: {'source': '/Users/iwamurahayato/myproject/0-tech-swat/astena-chatbot-mock/document/ey-japan-info-sensor-2023-06-03.pdf'}


In [ ]:
!pip install openai

In [ ]:
# 1. 必要なライブラリのインポート
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

print("ライブラリインポート完了")

# 2. OpenAI APIキーの設定（ローカル環境向けに手動）
import os
from openai import AzureOpenAI
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    endpoint="https://tech0-gpt-australiaeast.openai.azure.com/",
    credential=AzureKeyCredential("OPENAI_API_KEY")
)


# 3. 保存済みFAISSインデックスを読み込み
from langchain_community.vectorstores.faiss import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"
)

index = FAISS.load_local(
    folder_path=STORAGE_DIR,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
print(f"インデックス読み込み完了（{index.index.ntotal} 個のベクトル）")

# 4. プロンプトテンプレートの作成
question_prompt_template = """以下のコンテキスト情報を参照して、質問に日本語で答えてください。
コンテキストに関連する情報がない場合は、「提供された資料には該当する情報がありません」と回答してください。

コンテキスト:
{context}

質問: {question}

回答: """

QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template,
    input_variables=["context", "question"]
)
print("プロンプトテンプレート作成完了")

# 5. LLM（ChatGPT）設定
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.3,
    max_tokens=500
)
print("LLM設定完了")

# 6. RetrievalQA チェーン作成
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(search_kwargs={'k': 2}),
    chain_type_kwargs={"prompt": QUESTION_PROMPT},
    chain_type="stuff",
    return_source_documents=True
)
print("RetrievalQA チェーン作成完了")

# 7. テスト実行
test_questions = [
    "RAGとは何ですか？",
    "ファインチューニングとRAGの違いは？",
    "RAGの手順を教えてください"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n--- テスト {i} ---")
    print(f"質問: {question}")
    try:
        response = chain.invoke({"query": question})
        print(f"回答: {response['result']}")
        print("参照されたチャンク:")
        for j, source in enumerate(response["source_documents"], 1):
            print(f"   {j}. {source.page_content[:80]}...")
    except Exception as e:
        print(f"エラー: {e}")